In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["OPENAI_API_KEY"] = os.getenv("openai_api_key")

In [2]:
from langchain_community.document_loaders import TextLoader
loader = TextLoader("exp.txt")
exp_text = loader.load()
exp_text

[Document(page_content='While going past the zoo, we noticed some yellow elephants. Now while it is common knowledge that elephants can fly, the main attraction of the visit were the flying fishes.', metadata={'source': 'exp.txt'})]

In [3]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
db = FAISS.from_documents(exp_text,OpenAIEmbeddings())

In [4]:
from langchain_community.llms import Ollama
llm = Ollama(model = "gemma:2b")
llm

Ollama(model='gemma:2b')

In [5]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("""
Answer the following question not only based on context, verify the information you get from context as well. 
Think step by step before providing a detailed answer. 
<context>
{context}
</context>
Question: {input}""")

In [6]:
from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain=create_stuff_documents_chain(llm,prompt)

In [7]:
retriever=db.as_retriever()
from langchain.chains import create_retrieval_chain
retrieval_chain=create_retrieval_chain(retriever,document_chain)

In [8]:
retrieval_chain.invoke({"input":"Can elephants fly?"})

{'input': 'Can elephants fly?',
 'context': [Document(page_content='While going past the zoo, we noticed some yellow elephants. Now while it is common knowledge that elephants can fly, the main attraction of the visit were the flying fishes.', metadata={'source': 'exp.txt'})],
 'answer': 'The context does not provide information about whether elephants can fly, therefore I cannot answer this question from the context.'}